In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [2]:
df1 = pd.read_csv('ds_programming_customer_target_table.csv')
df2 = pd.read_csv('ds_programming_transaction_table.csv')

# Feature Engineering part

In [3]:
# get the transaction count 
txn_count = df2.groupby(['account_id'])['txn_id'].count().reset_index()
txn_count = txn_count.rename(columns={'txn_id': 'txn_count'})
txn_count

,account_id,txn_count
0,A000001,2
1,A000002,1
2,A000003,3
3,A000006,1
4,A000007,2
5,A000008,2
6,A000009,1
7,A000010,4
8,A000011,1
9,A000012,1


In [4]:
dfm1 = pd.merge(df1, txn_count, on='account_id' , how='left')

In [5]:
dfm1

,account_id,nric,latitude,longitude,is_active,txn_count
0,A000001,780805415511,3.131604,101.751612,1,2
1,A000002,941114105797,3.143432,101.741655,0,1
2,A000003,561101025240,3.157315,101.751012,1,3
3,A000006,790623085224,3.136490,101.747149,0,1
4,A000007,410313238221,3.159030,101.713245,1,2
5,A000008,440404440000,3.177542,101.750841,1,2
6,A000009,900622210013,3.177972,101.742602,0,1
7,A000010,311222100067,3.197683,101.743804,1,4
8,A000011,310222100288,3.146901,101.736940,0,1
9,A000012,660622210002,3.150557,101.708989,0,1


In [6]:
# Function to extract birth year and gender
def process_nric(nric):
    birth_year = int(nric[:2]) + 1900 # so it become 19XX 
    gender = 'Female' if int(nric[-1]) % 2 == 0 else 'Male' # last digit even number = Female else Male (odd digit)
    age = 2023 - birth_year # using 2023 - birth year to get current age
    return age, gender

In [7]:
# obtain the current age and gender by using the process_nric function
dfm1[['current_age', 'gender']] = dfm1['nric'].apply(lambda x: pd.Series(process_nric(str(x))))

In [8]:
# transform gender column with one-hot encoder
encoder = OneHotEncoder()
transformed = encoder.fit_transform(dfm1[['gender']])
dfm1[encoder.categories_[0]] = transformed.toarray()

In [9]:
dfm1

,account_id,nric,latitude,longitude,is_active,txn_count,current_age,gender,Female,Male
0,A000001,780805415511,3.131604,101.751612,1,2,45,Male,0.0,1.0
1,A000002,941114105797,3.143432,101.741655,0,1,29,Male,0.0,1.0
2,A000003,561101025240,3.157315,101.751012,1,3,67,Female,1.0,0.0
3,A000006,790623085224,3.136490,101.747149,0,1,44,Female,1.0,0.0
4,A000007,410313238221,3.159030,101.713245,1,2,82,Male,0.0,1.0
5,A000008,440404440000,3.177542,101.750841,1,2,79,Female,1.0,0.0
6,A000009,900622210013,3.177972,101.742602,0,1,33,Male,0.0,1.0
7,A000010,311222100067,3.197683,101.743804,1,4,92,Male,0.0,1.0
8,A000011,310222100288,3.146901,101.736940,0,1,92,Female,1.0,0.0
9,A000012,660622210002,3.150557,101.708989,0,1,57,Female,1.0,0.0


# Machine Learning part

In [10]:
# defining the features and target column 
X = dfm1[['latitude', 'longitude', 'txn_count', 'current_age', 'Female', 'Male']]
y = dfm1['is_active']

In [11]:
# split the dataset into training and testing sets
# using stratify to ensure the class will be balance during splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [12]:
# define the parameter grid for GridSearchCV (using GridSearch to perform hyperparameter tuning)
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 6, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [13]:
# perform GridSearchCV for the best parameter
rfc = RandomForestClassifier()
grid_search = GridSearchCV(rfc, param_grid, cv=2, scoring='accuracy')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=2, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 6, 7], 'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy')

In [14]:
# print the best parameters and the corresponding accuracy
print("Best Parameters: ", grid_search.best_params_)
print("Best Cross-Validated Accuracy: {:.2f}%".format(grid_search.best_score_*100))

Best Parameters:  {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Best Cross-Validated Accuracy: 100.00%


In [15]:
# evaluate the model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# display the classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



In [20]:
# create a DataFrame to display feature importances
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': best_model.feature_importances_
})

In [29]:
# sort the DataFrame by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# display the feature importances
print("Feature Importance:")
print(feature_importance_df)

Feature Importance:
       Feature  Importance
2    txn_count    0.285938
1    longitude    0.227774
3  current_age    0.207674
0     latitude    0.157567
4       Female    0.061309
5         Male    0.059739


In [30]:
# the classification report shows that in prediction set, 1 inactive user and 1 active user gets predicted correctly
# show the prediction output from the trained model
pred = best_model.predict(X)
print(pred)

[1 0 1 0 1 1 0 1 0 0]


In [36]:
# expected output
result = pd.DataFrame({'account_id': dfm1['account_id'], 'is_active': pred})
result

,account_id,is_active
0,A000001,1
1,A000002,0
2,A000003,1
3,A000006,0
4,A000007,1
5,A000008,1
6,A000009,0
7,A000010,1
8,A000011,0
9,A000012,0


In [37]:
df_active = dfm1.loc[dfm1.is_active==1]
df_inactive = dfm1.loc[dfm1.is_active==0]

In [38]:
print("mean comparison:", df_active['current_age'].mean(), df_inactive['current_age'].mean())
print("median comparison:", df_active['current_age'].median(), df_inactive['current_age'].median())

mean comparison: 73.0 51.0
median comparison: 79.0 44.0


In [39]:
print("female count:", df_active['Female'].sum(), df_inactive['Female'].sum())
print("male count: ", df_active['Male'].sum(), df_inactive['Male'].sum())

female count: 2.0 3.0
male count:  3.0 2.0


Additional analysis:

- the most important feature is txn_count with the importance of 0.285938; the higher the txn_count, the user is more likely to be an active user and inactive user has a maximum transaction count of 1
- the 5 predicted active users in the dataframe age ranged from 45-92, which has a higher mean age (73) & median age (79) of compared to inactive users mean age (51) & median age (44); and current age is the no 3 most importance factor among all 6 factors
- the gender comparison for both active and inactive user are not distinctive (quite balanced):
    - Active group has 2 female and 3 male
    - Inactive group has 3 female and 2 male
    - this explains why 'gender' column is the least important feature (bottom 2 in feature importance table; Female - 0.061309 and Male - 0.059739)